This notebook demonstrates a test of refining the fundamental matrix using the slopes between keypoints.

In [1]:
import os
import pandas as pd
import sys
sys.path.insert(0, os.path.abspath('..'))

import unittest

from autocnet.examples import get_path
from autocnet.fileio.io_controlnetwork import to_isis
from autocnet.fileio.io_controlnetwork import write_filelist
from autocnet.graph.network import CandidateGraph
from autocnet.fileio import utils

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
serial_numbers = {'AS15-M-0295_SML.png': '1971-07-31T01:24:11.754',
                'AS15-M-0296_SML.png': '1971-07-31T01:24:36.970',
                'AS15-M-0297_SML.png': '1971-07-31T01:25:02.243',
                'AS15-M-0298_SML.png': '1971-07-31T01:25:27.457',
                'AS15-M-0299_SML.png': '1971-07-31T01:25:52.669',
                'AS15-M-0300_SML.png': '1971-07-31T01:26:17.923'}

for k, v in serial_numbers.items():
    serial_numbers[k] = 'APOLLO15/METRIC/{}'.format(v)

In [3]:
adjacency = get_path('two_image_adjacency.json')
basepath = get_path('Apollo15')
cg = CandidateGraph.from_adjacency(adjacency, basepath=basepath)

cg.extract_features(method='sift', extractor_parameters={"nfeatures":500})
cg.match_features(k=2)

In [4]:
for source, destination, edge in cg.edges_iter(data=True):
    # Perform the symmetry check
    edge.symmetry_check()
    # Perform the ratio test
    edge.ratio_check(clean_keys=['symmetry'])

/home/kberry/anaconda3/envs/autocnet_new/lib/python3.5/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [5]:
cg.apply_func_to_edges("compute_fundamental_matrix", clean_keys=['symmetry', 'ratio'])

In [6]:
f = cg.edge[0][1].fundamental_matrix

In [7]:
df=utils.calculate_slope(f.x1, f.x2)
len(df.values.ravel())

75

In [8]:
f.mask[f.mask==True].equals(f.mask[f.mask])

True

In [9]:
nbr_inliers = 20
fp = np.array(np.random.standard_normal((nbr_inliers, 2)))  # inliers
static_F = np.array([[4, 0.5, 10], [0.25, 1, 5], [0.2, 0.1, 1]])
fph = np.hstack((fp, np.ones((nbr_inliers, 1))))
tp = static_F.dot(fph.T)
# normalize hom. coordinates
tp /= tp[-1, :np.newaxis]
df1 = pd.DataFrame(fp, columns=['x', 'y'])
df2 = pd.DataFrame(tp.T[:, :2], columns=['x', 'y'])

In [10]:
df1

,x,y
0,1.154769,-0.135218
1,-0.188116,-0.636273
2,1.054127,0.551156
3,-0.468411,1.003616
4,-0.334554,-0.059205
5,0.462206,-1.910069
6,0.691400,0.526320
7,0.888097,0.146263
8,2.102747,0.474393
9,0.453846,-0.138034


In [11]:
df2

,x,y
0,11.952591,4.233070
1,9.935361,4.803004
2,11.447679,4.593174
3,8.570915,5.847456
4,9.310261,5.238697
5,12.084951,3.555980
6,10.940153,4.785551
7,11.428450,4.502669
8,12.703221,4.087279
9,10.906909,4.619856


In [12]:
mask = df1.x > 0
mask

0      True
1     False
2      True
3     False
4     False
5      True
6      True
7      True
8      True
9      True
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17     True
18    False
19     True
Name: x, dtype: bool

In [13]:
df1[mask]

,x,y
0,1.154769,-0.135218
2,1.054127,0.551156
5,0.462206,-1.910069
6,0.691400,0.526320
7,0.888097,0.146263
8,2.102747,0.474393
9,0.453846,-0.138034
17,1.485067,0.748019
19,1.854610,0.538104


In [14]:
df2[mask]

,x,y
0,11.952591,4.233070
2,11.447679,4.593174
5,12.084951,3.555980
6,10.940153,4.785551
7,11.428450,4.502669
8,12.703221,4.087279
9,10.906909,4.619856
17,11.892474,4.460721
19,12.414606,4.212550


In [15]:
df1[mask] = df2[mask]

In [16]:
df1

,x,y
0,11.952591,4.233070
1,-0.188116,-0.636273
2,11.447679,4.593174
3,-0.468411,1.003616
4,-0.334554,-0.059205
5,12.084951,3.555980
6,10.940153,4.785551
7,11.428450,4.502669
8,12.703221,4.087279
9,10.906909,4.619856


In [ ]:
f.refine(values=utils.calculate_slope(f.x1, f.x2).values)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [ ]:
f.rollback()

In [ ]:
f.mask

In [ ]:
figsize(18,18)
fig, ax = plt.subplots(1,1)
ax = cg.edge[0][1].plot(clean_keys=['ratio', 'fundamental'], ax=ax)

In [ ]:
figsize(18,18)
fig, ax = plt.subplots(1,1)
ax = cg.edge[0][2].plot(clean_keys=['ratio', 'symmetry', 'fundamental', 'subpixel'], ax=ax)